In this project you will use and modify the module you developed previously (File Encryption). You'll also use the [OS package](https://docs.python.org/3/library/os.html) as well as [JSON](https://docs.python.org/3/library/json.html) package.

Step 1:

Modify your File Encryption to include the policy of Encrypt-then-[MAC](https://cryptography.io/en/latest/hazmat/primitives/mac/hmac/) for every encryption.

(C, IV, tag)= **MyencryptMAC**(message, EncKey, HMACKey)

(C, IV, tag, Enckey, HMACKey, ext)= **MyfileEncryptMAC** (filepath)

(RSACipher, C, IV, tag, ext)= **MyRSAEncrypt**(filepath, RSA_Publickey_filepath)

Where in the above, RSACipher includes RSA encryption of m=EncKey+ HMACKey (concatenated). Make sure you implement the reverse of the above (MACVerification-then-Decrypt). You can use SHA256 in your [HMAC](https://cryptography.io/en/latest/hazmat/primitives/mac/hmac/).

Save this class as FileEncryptMAC.

Step 2:

Next, you will a script that looks for a pair of RSA Public and private key (using a CONSTANT file path; PEM format). If the files do not exist (use OS package) then generate the RSA public and private key (2048 bits length) using the same constant file path.

Step 3:

You can use the OS package to retrieve the current working directory. Then you can get a list of all files in this directory. For each file, encrypt them using **MyRSAEncrypt** from your new FileEncryptMAC module. Do this in a loop for all files (make sure you do not encrypt the RSA Private Key file). For every file that is encrypted, store the encrypted file as a JSON file. The attributes you have for each file are 'RSACipher', 'C', 'IV', 'tag' and 'ext'. The values are from MyRSAEncrypt method. Once the JSON fire is written (use json.dump() with file.write() methods) into a JSON file then you can remove the plaintext file (use os.remove() method).

Note: For now, you can skip encrypting files within directories in the working directories (i.e., recursive execution).

Note: DO NOT test your script on any valuable file. It will be your responsibility if you lose any important data to you.

Step 4:

Using [Pyinstaller](http://www.pyinstaller.org/) or [Py2exe](http://www.py2exe.org/) create an executable file from your step 3.

Do NOT run the executable file on important folders. Only test on a designated python working directory. You are responsible if you lose any important file.

## (C, IV, tag)= MyencryptMAC(message, EncKey, HMACKey)

In [3]:
import os
import sys
import base64
import cryptography
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, serialization, hashes, asymmetric as asymm, hashes, hmac
from cryptography.hazmat.primitives.asymmetric import rsa

# encryption method AES-CBC-256, Encrypt-then-MAC(SHA256)
def MyencryptMAC(message, EncKey, HMACKey):
    
    #--- STEP 1: Encrypt ---
    
    #check key length
    if len(EncKey)<32 or len(HMACKey)<32:
        print("Error: key lengthis less than 32 bytes.")
        return
    
    # convert message to bytes. Pass if message is already bytes type.
    try:
        message = message.encode()#convert string to bytes
    except:
        pass
    
    # encrypt a message
    try:           
        IV = os.urandom(16)   #generate an Initialization vector

        # pad the message
        padder = padding.PKCS7(128).padder()
        padded_data = padder.update(message) + padder.finalize()
        message = padded_data

        #calling the default AES CBC mode
        cipher = Cipher(algorithms.AES(EncKey), modes.CBC(IV), backend=default_backend())
        #creating an encryptor object
        encryptor = cipher.encryptor()
        #generating cipher text
        C = encryptor.update(message) + encryptor.finalize()

        print("Success: Encryption finished.")        
    except:
        print("Error: Encryption failed.")
        return
    

    #--- STEP 2: HMAC ---
    try:        
        # create a tag
        tag = hmac.HMAC(HMACKey, hashes.SHA256(), backend=default_backend())
        tag.update(C)
        tag = tag.finalize()
        
        print("Success: HMAC tag finished.")
        print("MyencryptMAC complete.")
        return(C, IV, tag)
    except:
        print("Error: HMAC tag failed.")
        return
     


# MACVerification-then-Decrypt
def MydecryptMAC(C, IV, tag, EncKey, HMACKey):

    #Step 1: Verify
    # create HMAC_tag
    h = hmac.HMAC(HMACKey, hashes.SHA256(), backend=default_backend())
    h.update(C)
    h = h.finalize()
    
    # compare HMAC_tag with tag belongs to encrypted content
    if h == tag:
        print("Success: Tag verified.")
        cipher = Cipher(algorithms.AES(EncKey), modes.CBC(IV), backend=default_backend())
        #creating a decryptor object
        decryptor = cipher.decryptor()
        plaintext = decryptor.update(C) + decryptor.finalize()

        try:
            #unpad the plaintext
            unpadder = padding.PKCS7(128).unpadder()
            pt = unpadder.update(plaintext) + unpadder.finalize()
            print("MydecryptMAC Complete.\nf")
            return plaintext
        except:
            #plaintext does not require unpadding
            print("MydecryptMAC Complete.\n")
            return plaintext

    else:
        print("Error: Tag verified failed.\n")
        return

    
    
# #-------------------------
# # execution code    
# EncKey = os.urandom(32)
# HMACKey = os.urandom(32)
# m = "a secret message"

# try:
#     C, IV, tag = MyencryptMAC(m, EncKey, HMACKey)
#     print("\nEncrypted Message with a tag:\n")
#     print(C,"\n", IV, "\n", tag)
# except:
#     print("Error: MyencryptMAC failed.\n")
    

# try:
#     print("\nDecrypted Message:\n")
#     print(MydecryptMAC(C, IV, tag, EncKey, HMACKey))
# except:
#     print("Error: MydecryptMAC failed.\n")

## (C, IV, tag, Enckey, HMACKey, ext)= **MyfileEncryptMAC** (filepath)


In [61]:
# file encryption algorithm
def MyfileEncryptMAC(filepath):
    
    try:
        #generate keys
        EncKey = os.urandom(32)
        HMACKey = os.urandom(32)
    except:
        print("NameError: name 'os' is not defined")
        return
    
    try:
        # Read the entire file as a single byte string
        with open(filepath, 'rb') as f:
            data = f.read()
    except:
        print("Error: filepath is invalid.")
        return
    
    try:       
        try:
            result = MyencryptMAC(data, EncKey, HMACKey)
            print("Success: Encrypted file with a tag.")
        except:
            print("Error: MyencryptMAC failed.")
                
        ext = os.path.splitext(file_path)[1]
        result += (EncKey, HMACKey, ext)
        input_enc_file = input("Enter a name for an output encrypted file without extension (e.g. \"encrypted_image\"): ")

        # create a writable image and write the decoding result
        input_enc_filepath = os.path.abspath(input_enc_file+ext)
        image_result = open(input_enc_filepath, 'wb') 
        image_result.write(result[0])
        print("Complete: Encrypted file named \"{}\".\n".format(input_enc_file+ext))
        return result
    except:
        print("Error: MyfileEncryptMAC failed.\n")
        return


# file dencryption algorithm
def MyfileDecryptMAC(enc_filepath, iv, tag, key, h, ext):

    with open(enc_filepath, 'rb') as f:
        data = f.read()
    
    try:
        plaintext = MydecryptMAC(data, iv, tag, key, h)
        print("Success: Decrypted file with a tag.")
    except:
        print("Error: MydecryptMAC failed.")
    
    dec_file_name = input("Enter a name for an output decrypted file without extension (e.g. \"decrypted_image\"): ")
    dec_file_path = os.path.abspath(dec_file_name + ext)
    image_result = open(dec_file_path, 'wb') # create a writable image and write the decoding result
    image_result.write(plaintext)
    print("Complete: Decrypted file named \"{}\".\n".format(dec_file_name + ext))



# # ----------------------
# #Execution code
    
# print("MyfileEncryptMAC...running")
# file_name = input("Enter a file with extension for encryption(e.g. \"hello.png\"): ")
# file_path = os.path.abspath(file_name)
# print(file_path)
# try:
#     C, IV, tag, Enckey, HMACKey, ext = MyfileEncryptMAC(file_path)
# except:
#     print("Error: MyfileEncryptMAC failed.\n")


# print("MyfileDecryptMAC...running")
# input_enc_file = input("Enter an encrypted file's name without extension: ")
# input_enc_filepath = os.path.abspath(input_enc_file+ext)
# try:
#     MyfileDecryptMAC(input_enc_filepath, IV, tag, Enckey, HMACKey, ext)
# except:
#     print("Error: MyfileDecryptMAC failed.\n")

## (RSACipher, C, IV, tag, ext)= **MyRSAEncrypt**(filepath, RSA_Publickey_filepath)

In [72]:
#create public/private key pair
def create_pem_key_pair():
    # create key object
    backend = default_backend()
    key = rsa.generate_private_key(backend=backend, public_exponent=65537,key_size=2048)
    
    # private key
    private_key = key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
            )
    with open("private.pem", 'wb') as private_pem:
        private_pem.write(private_key)
        private_pem.close()
    
    #public key
    public_key = key.public_key().public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
    with open("public.pem", 'wb') as public_pem:
        public_pem.write(public_key)
        public_pem.close()
        
    print("Success: Created \"public.pem\" and \"private.pem\" ")
    
    
#RSA encryption method
def MyRSAEncryptMAC(file_path, RSA_Publickey_filepath):
    #encrypting an image file
    print("MyfileEncryptMAC...running")
    try:
        C, IV, tag, Enckey, HMACKey, ext = MyfileEncryptMAC(file_path)
    except:
        print("Error: MyfileEncryptMAC failed.\n")
        return
    
    try:
        with open(RSA_Publickey_filepath, "rb") as p_key:
            public_key = serialization.load_pem_public_key(p_key.read(),backend=default_backend())
    except:
        print("Error: Pem Public key's filepath is invalid.")
        return
    
    #obtain RSACipher
    try:
        RSACipher = public_key.encrypt(Enckey+ "break".encode() +HMACKey, asymm.padding.OAEP(
                                               mgf=asymm.padding.MGF1(algorithm=hashes.SHA256()),
                                               algorithm=hashes.SHA256(),
                                               label=None ))
    except:
        print("Error: RSACipher cannot be generated.")
        return
        
    return RSACipher, C, IV, tag, ext



#RSA decryption method
def MyRSADecryptMAC(RSACipher, enc_file_path, IV, tag, ext, RSA_Privatekey_filepath):

    try:
        with open(RSA_Privatekey_filepath, "rb") as Enckey:
            private_key = serialization.load_pem_private_key(Enckey.read(),password=None, backend=default_backend())
    except:
        print("Error: Pem private key's filepath is invalid.")
            
    try:
        key = private_key.decrypt(
            RSACipher,
            asymm.padding.OAEP(mgf=asymm.padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
    
        # split the Enc_key and HMAC_key
        key_list = key.split("break".encode())
        Enckey, HMACKey = key_list[0], key_list[1]
    except:
        print("Error: Enc_key and HMAC_key cannot be generated from RSACipher.")
        
        
    print("MyfileDecryptMAC...running")
    try:
        MyfileDecryptMAC(enc_file_path, IV, tag, Enckey, HMACKey, ext)
    except:
        print("Error: MyfileDecryptMAC failed.\n")


    
    
# #----------------------
# #execution code

# create_pem_key_pair() 
# filename = input("Enter a file with extension for encryption(e.g. \"hello.png\"): ")
# filepath = os.path.abspath(filename)

# public_pem = input("Enter pem public key file name with extension (e.g. \"public.pem\"): ")
# public_pem_path = os.path.abspath(public_pem)
# try:
#     RSACipher, C, IV, tag, ext = MyRSAEncryptMAC(filepath, public_pem_path)
# except:
#     print("Error: MyRSAEncryptMAC failed.")

    
# enc_filename = input("Enter a file without extension for encryption(e.g. \"encrypted_image\"): ")
# enc_filepath = os.path.abspath(enc_filename)
# private_pem = input("Enter pem private key file name with extension (e.g. \"private.pem\"): ")
# private_pem_path = os.path.abspath(private_pem)

# try:
#     enc_filepath = os.path.abspath(enc_filename+ext)
#     MyRSADecryptMAC(RSACipher, enc_filepath, IV, tag, ext, private_pem_path)
# except:
#     print("Error: MyRSADecryptMAC failed.")

Success: Created "public.pem" and "private.pem" 
Enter a file with extension for encryption(e.g. "hello.png"): hello.png
Enter pem public key file name with extension (e.g. "public.pem"): public.pem
MyfileEncryptMAC...running
Success: Encryption finished.
Success: HMAC tag finished.
MyencryptMAC complete.
Success: Encrypted file with a tag.
Enter a name for an output encrypted file without extension (e.g. "encrypted_image"): f
Complete: Encrypted file named "f.png".

b"\xbe]\x99i\x95\xc7\xfe\x1e\xaf\x96\x9d'\x9cn\xa3!\xfd\x04\xdb}\x82\x8e\xfbQ'_\xb0\xa7A\xc1*5"
b'\xb9\xe6\xe9\xf2\xeap\x94\xf2\x95\xed\xdd\xd156\xb4x\xd5\xc5\xa9*\n\x87\x91\xa3\x05\xcfq\xdd\x977\x8ab'
Enter a file without extension for encryption(e.g. "encrypted_image"): f
Enter pem private key file name with extension (e.g. "private.pem"): private.pem
b"\xbe]\x99i\x95\xc7\xfe\x1e\xaf\x96\x9d'\x9cn\xa3!\xfd\x04\xdb}\x82\x8e\xfbQ'_\xb0\xa7A\xc1*5"
b'\xb9\xe6\xe9\xf2\xeap\x94\xf2\x95\xed\xdd\xd156\xb4x\xd5\xc5\xa9*\n\x87\x9

## "FileEncryptMAC" class

In [73]:
import os
import sys
import base64
import cryptography
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, serialization, hashes, asymmetric as asymm, hashes, hmac
from cryptography.hazmat.primitives.asymmetric import rsa

class FileEncryptMAC:

    #--- Part 1: (C, IV, tag)= MyencryptMAC(message, EncKey, HMACKey) ---
    
    # encryption method AES-CBC-256, Encrypt-then-MAC(SHA256)
    def MyencryptMAC(message, EncKey, HMACKey):

        #--- STEP 1: Encrypt ---

        #check key length
        if len(EncKey)<32 or len(HMACKey)<32:
            print("Error: key lengthis less than 32 bytes.")
            return

        # convert message to bytes. Pass if message is already bytes type.
        try:
            message = message.encode()#convert string to bytes
        except:
            pass

        # encrypt a message
        try:           
            IV = os.urandom(16)   #generate an Initialization vector

            # pad the message
            padder = padding.PKCS7(128).padder()
            padded_data = padder.update(message) + padder.finalize()
            message = padded_data

            #calling the default AES CBC mode
            cipher = Cipher(algorithms.AES(EncKey), modes.CBC(IV), backend=default_backend())
            #creating an encryptor object
            encryptor = cipher.encryptor()
            #generating cipher text
            C = encryptor.update(message) + encryptor.finalize()

            print("Success: Encryption finished.")        
        except:
            print("Error: Encryption failed.")
            return


        #--- STEP 2: HMAC ---
        try:        
            # create a tag
            tag = hmac.HMAC(HMACKey, hashes.SHA256(), backend=default_backend())
            tag.update(C)
            tag = tag.finalize()

            print("Success: HMAC tag finished.")
            print("MyencryptMAC complete.")
            return(C, IV, tag)
        except:
            print("Error: HMAC tag failed.")
            return



    # MACVerification-then-Decrypt
    def MydecryptMAC(C, IV, tag, EncKey, HMACKey):

        #Step 1: Verify
        # create HMAC_tag
        h = hmac.HMAC(HMACKey, hashes.SHA256(), backend=default_backend())
        h.update(C)
        h = h.finalize()

        # compare HMAC_tag with tag belongs to encrypted content
        if h == tag:
            print("Success: Tag verified.")
            cipher = Cipher(algorithms.AES(EncKey), modes.CBC(IV), backend=default_backend())
            #creating a decryptor object
            decryptor = cipher.decryptor()
            plaintext = decryptor.update(C) + decryptor.finalize()

            try:
                #unpad the plaintext
                unpadder = padding.PKCS7(128).unpadder()
                pt = unpadder.update(plaintext) + unpadder.finalize()
                print("MydecryptMAC Complete.\nf")
                return plaintext
            except:
                #plaintext does not require unpadding
                print("MydecryptMAC Complete.\n")
                return plaintext

        else:
            print("Error: Tag verified failed.\n")
            return

        
    #--- Part 2: (C, IV, tag, Enckey, HMACKey, ext)= MyfileEncryptMAC(filepath) ---
    def MyfileEncryptMAC(filepath):

        try:
            #generate keys
            EncKey = os.urandom(32)
            HMACKey = os.urandom(32)
        except:
            print("NameError: name 'os' is not defined")
            return

        try:
            # Read the entire file as a single byte string
            with open(filepath, 'rb') as f:
                data = f.read()
        except:
            print("Error: filepath is invalid.")
            return

        try:       
            try:
                result = MyencryptMAC(data, EncKey, HMACKey)
                print("Success: Encrypted file with a tag.")
            except:
                print("Error: MyencryptMAC failed.")

            ext = os.path.splitext(file_path)[1]
            result += (EncKey, HMACKey, ext)
            input_enc_file = input("Enter a name for an output encrypted file without extension (e.g. \"encrypted_image\"): ")

            # create a writable image and write the decoding result
            input_enc_filepath = os.path.abspath(input_enc_file+ext)
            image_result = open(input_enc_filepath, 'wb') 
            image_result.write(result[0])
            print("Complete: Encrypted file named \"{}\".\n".format(input_enc_file+ext))
            return result
        except:
            print("Error: MyfileEncryptMAC failed.\n")
            return


    # file dencryption algorithm
    def MyfileDecryptMAC(enc_filepath, iv, tag, key, h, ext):

        with open(enc_filepath, 'rb') as f:
            data = f.read()

        try:
            plaintext = MydecryptMAC(data, iv, tag, key, h)
            print("Success: Decrypted file with a tag.")
        except:
            print("Error: MydecryptMAC failed.")

        dec_file_name = input("Enter a name for an output decrypted file without extension (e.g. \"decrypted_image\"): ")
        dec_file_path = os.path.abspath(dec_file_name + ext)
        image_result = open(dec_file_path, 'wb') # create a writable image and write the decoding result
        image_result.write(plaintext)
        print("Complete: Decrypted file named \"{}\".\n".format(dec_file_name + ext))

        
    #--- Part 3:  (RSACipher, C, IV, tag, ext)= MyRSAEncrypt(filepath, RSA_Publickey_filepath) ---

    #create public/private key pair
    def create_pem_key_pair():
        # create key object
        backend = default_backend()
        key = rsa.generate_private_key(backend=backend, public_exponent=65537,key_size=2048)

        # private key
        private_key = key.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
                )
        with open("private.pem", 'wb') as private_pem:
            private_pem.write(private_key)
            private_pem.close()

        #public key
        public_key = key.public_key().public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
                )
        with open("public.pem", 'wb') as public_pem:
            public_pem.write(public_key)
            public_pem.close()

        print("Success: Created \"public.pem\" and \"private.pem\" ")


    #RSA encryption method
    def MyRSAEncryptMAC(file_path, RSA_Publickey_filepath):
        #encrypting an image file
        print("MyfileEncryptMAC...running")
        try:
            C, IV, tag, Enckey, HMACKey, ext = MyfileEncryptMAC(file_path)
        except:
            print("Error: MyfileEncryptMAC failed.\n")
            return

        try:
            with open(RSA_Publickey_filepath, "rb") as p_key:
                public_key = serialization.load_pem_public_key(p_key.read(),backend=default_backend())
        except:
            print("Error: Pem Public key's filepath is invalid.")
            return

        #obtain RSACipher
        try:
            RSACipher = public_key.encrypt(Enckey+ "break".encode() +HMACKey, asymm.padding.OAEP(
                                                   mgf=asymm.padding.MGF1(algorithm=hashes.SHA256()),
                                                   algorithm=hashes.SHA256(),
                                                   label=None ))
        except:
            print("Error: RSACipher cannot be generated.")
            return

        return RSACipher, C, IV, tag, ext



    #RSA decryption method
    def MyRSADecryptMAC(RSACipher, enc_file_path, IV, tag, ext, RSA_Privatekey_filepath):

        try:
            with open(RSA_Privatekey_filepath, "rb") as Enckey:
                private_key = serialization.load_pem_private_key(Enckey.read(),password=None, backend=default_backend())
        except:
            print("Error: Pem private key's filepath is invalid.")

        try:
            key = private_key.decrypt(
                RSACipher,
                asymm.padding.OAEP(mgf=asymm.padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))

            # split the Enc_key and HMAC_key
            key_list = key.split("break".encode())
            Enckey, HMACKey = key_list[0], key_list[1]
        except:
            print("Error: Enc_key and HMAC_key cannot be generated from RSACipher.")


        print("MyfileDecryptMAC...running")
        try:
            MyfileDecryptMAC(enc_file_path, IV, tag, Enckey, HMACKey, ext)
        except:
            print("Error: MyfileDecryptMAC failed.\n")


In [ ]:
#----------------------
#execution code

create_pem_key_pair() 
filename = input("Enter a file with extension for encryption(e.g. \"hello.png\"): ")
filepath = os.path.abspath(filename)

public_pem = input("Enter pem public key file name with extension (e.g. \"public.pem\"): ")
public_pem_path = os.path.abspath(public_pem)
try:
    RSACipher, C, IV, tag, ext = MyRSAEncryptMAC(filepath, public_pem_path)
except:
    print("Error: MyRSAEncryptMAC failed.")

    
enc_filename = input("Enter a file without extension for encryption(e.g. \"encrypted_image\"): ")
enc_filepath = os.path.abspath(enc_filename)
private_pem = input("Enter pem private key file name with extension (e.g. \"private.pem\"): ")
private_pem_path = os.path.abspath(private_pem)

try:
    enc_filepath = os.path.abspath(enc_filename+ext)
    MyRSADecryptMAC(RSACipher, enc_filepath, IV, tag, ext, private_pem_path)
except:
    print("Error: MyRSADecryptMAC failed.")

## testing

In [1]:
    #--- Part 2: (C, IV, tag, Enckey, HMACKey, ext)= MyfileEncryptMAC(filepath) ---
    def MyfileEncryptMAC(self, filepath):
        
        try:
            #generate keys
            EncKey = os.urandom(32)
            HMACKey = os.urandom(32)
        except:
            print("NameError: name 'os' is not defined")
            return

        try:
            # Read the entire file as a single byte string
            with open(filepath, 'rb') as f:
                data = f.read()
        except:
            print("Error: filepath is invalid.")
            return

        try:       
            try:
                result = self.MyencryptMAC(data, EncKey, HMACKey)
                print("Success: Encrypted file with a tag.")
            except:
                print("Error: MyencryptMAC failed.")
            
            ext = os.path.splitext(filepath)[1]
            result += (EncKey, HMACKey, ext)
            input_enc_file = input("Enter a name for an output encrypted file without extension (e.g. \"encrypted_image\"): ")

            # create a writable image and write the decoding result
            input_enc_filepath = os.path.abspath(input_enc_file+ext)
            image_result = open(input_enc_filepath, 'wb') 
            image_result.write(result[0])         
            print("Complete: Encrypted file named \"{}\".\n".format(input_enc_file+ext))
            return result
        except:
            print("Error: MyfileEncryptMAC failed.\n")
            return

In [4]:


# ----------------------
#Execution code
    
print("MyfileEncryptMAC...running")
file_path = os.path.abspath("hello.png")
print(file_path)
try:
    C, IV, tag, Enckey, HMACKey, ext = MyfileEncryptMAC(file_path)
except:
    print("Error: MyfileEncryptMAC failed.\n")


MyfileEncryptMAC...running
/Users/adrian/Documents/Work_School/GroupNumberOne/3. RSA File/hello.png
Error: MyfileEncryptMAC failed.

